<a href="https://colab.research.google.com/github/abiisikumbang/machinelearning/blob/main/Credit_Risk_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LOAD DATA

In [75]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

In [76]:
os.listdir()

['.config',
 'lc_loan.csv',
 'us-state-codes.csv',
 'new.html',
 '.ipynb_checkpoints',
 'lc_2016_2017.csv',
 'sample_data']

In [77]:
loan_data = pd.read_csv('lc_2016_2017.csv')
loan_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,112435993,NaN,2300,2300,2300.0,36 months,12.62,77.08,C,C1,...,0.0,NaN,1.0,2.0,2315.0,55.0,7100.0,1.0,0.0,2.0
1,112290210,NaN,16000,16000,16000.0,60 months,12.62,360.95,C,C1,...,11078.0,69.0,3.0,5.0,1962.0,94.0,31900.0,0.0,6.0,1.0
2,112436985,NaN,6025,6025,6025.0,36 months,15.05,209.01,C,C4,...,0.0,NaN,1.0,2.0,1950.0,45.0,27700.0,1.0,5.0,3.0
3,112439006,NaN,20400,20400,20400.0,36 months,9.44,652.91,B,B1,...,53566.0,95.0,1.0,2.0,4240.0,60.0,46900.0,1.0,1.0,1.0
4,112438929,NaN,13000,13000,13000.0,36 months,11.99,431.73,B,B5,...,8466.0,72.0,0.0,1.0,2996.0,78.0,7800.0,0.0,0.0,0.0


# CHECK DATASET

In [78]:
#check shape dan describe
loan_data.shape

(759338, 72)

In [79]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759338 entries, 0 to 759337
Data columns (total 72 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           759338 non-null  int64  
 1   member_id                    0 non-null       float64
 2   loan_amnt                    759338 non-null  int64  
 3   funded_amnt                  759338 non-null  int64  
 4   funded_amnt_inv              759338 non-null  float64
 5   term                         759338 non-null  object 
 6   int_rate                     759338 non-null  float64
 7   installment                  759338 non-null  float64
 8   grade                        759338 non-null  object 
 9   sub_grade                    759338 non-null  object 
 10  emp_title                    708368 non-null  object 
 11  emp_length                   708975 non-null  object 
 12  home_ownership               759338 non-null  object 
 13 

In [80]:
#cek value dari load_status
loan_data.loan_status.value_counts()

,count
loan_status,
Current,565523
Fully Paid,130718
Charged Off,37197
Late (31-120 days),15354
In Grace Period,6634
Late (16-30 days),3876
Default,36


In [81]:
#melihat unique values pada feature loan_status
loan_data.loan_status.unique()

array(['Current', 'Fully Paid', 'In Grace Period', 'Late (31-120 days)',
       'Late (16-30 days)', 'Charged Off', 'Default'], dtype=object)

In [82]:
# Membuat feature baru yaitu good_bad sebagai target variable,
# Jika loan_statusnya 'Charged Off', 'Default', 'Late (31-120 days)', 'Late (16-30 days)'
# akan dianggap sebagai bad_loan atau 1 dan nilai selain itu akan dianggap good loan atau 0
loan_data['good_bad'] = np.where(loan_data.loc[:, 'loan_status'].isin(['Charged Off',
                                                                   'Default', 'Late (31-120 days)', 'Late (16-30 days)'])
                                , 1 , 0)

In [83]:
loan_data.good_bad.value_counts()

,count
good_bad,
0,702875
1,56463


In [84]:
loan_data.good_bad.value_counts(normalize=True)

,proportion
good_bad,
0,0.925642
1,0.074358


# CLEANING DATA

Feature yang memiliki missing values lebih dari 50% akan di drop, karena jika ingin diisi dengan nilai lain seperti median atau mean, maka errornya akan sangat tinggi. Lebih baik di drop agar tidak membuat model semakin tidak akurat.

In [85]:
# Melihat feature apa saja yang memiliki missing value lebih dari 50%

#meihat missing value dengan presentase
missing_values = pd.DataFrame(loan_data.isnull().sum()/loan_data.shape[0])

#memfilter missing value >50% dan mengurutkannya dari yang terbesar
missing_values = missing_values[missing_values.iloc[:,0] > 0.50]
missing_values.sort_values([0], ascending=False)

,0
member_id,1.000000
desc,0.999978
dti_joint,0.955223
annual_inc_joint,0.955220
verification_status_joint,0.955220
mths_since_last_record,0.814076
mths_since_last_major_derog,0.716720


In [86]:
#menghapus data yang missing value >50%
loan_data.dropna(thresh = loan_data.shape[0]*0.5, axis=1, inplace=True)

In [87]:
loan_data.isnull().sum()

,0
id,0
loan_amnt,0
funded_amnt,0
funded_amnt_inv,0
term,0
...,...
total_rev_hi_lim,0
inq_fi,61
total_cu_tl,62
inq_last_12m,62


#DATA SPLITTING


In [88]:
loan_data.shape

(759338, 66)

In [89]:
from sklearn.model_selection import train_test_split

In [90]:
# Membagi data menjadi 80/20 dengan menyamakan distribusi dari bad loans di test set dengan train set.
X = loan_data.drop('good_bad', axis=1)
y = loan_data['good_bad']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, stratify= y, random_state=42)

In [91]:
y_train.value_counts(normalize=True)

,proportion
good_bad,
0,0.925642
1,0.074358


In [92]:
# Distribusi y_test sudah sama persis dengan y_train
y_test.value_counts(normalize=True)

,proportion
good_bad,
0,0.925639
1,0.074361


ARTINYA KASUSNYA IMBALANCE KARNA KELAS 1 KURANG DARI 1%, CARA MENANGANI NYA HARUS MEMILIKI DISTRIBUSI YANG SAMA

# DATA CLEANING


In [93]:
X_train.shape

(607470, 65)

In [94]:
#melihat kolom yang bertipe objek dan booleyan pada data x_train
X_train.select_dtypes(['object', 'bool']).columns

Index(['term', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state',
       'earliest_cr_line', 'initial_list_status', 'last_pymnt_d',
       'next_pymnt_d', 'last_credit_pull_d', 'application_type'],
      dtype='object')

In [95]:
 for col in X_train.select_dtypes(include= ['object','bool']).columns:
     print(col)
     print(X_train[col].unique())
     print()

term
['36 months' '60 months']

grade
['C' 'A' 'B' 'E' 'D' 'G' 'F']

sub_grade
['C1' 'A1' 'B1' 'B5' 'A4' 'E2' 'A5' 'D1' 'C4' 'C3' 'G3' 'D3' 'E1' 'C5'
 'D2' 'C2' 'A2' 'E3' 'D5' 'E4' 'B2' 'F2' 'B4' 'A3' 'D4' 'B3' 'F1' 'F5'
 'G2' 'E5' 'F3' 'G1' 'G5' 'F4' 'G4']

emp_title
['Administrative assistant' 'director of workforce development' 'Nanny'
 ... 'Operations Planning Manager' 'Sr. xPression Developer'
 'making plant planner']

emp_length
['4 years' '10+ years' '< 1 year' '5 years' '6 years' '7 years' '2 years'
 '3 years' '8 years' '1 year' nan '9 years']

home_ownership
['MORTGAGE' 'RENT' 'OWN' 'ANY' 'NONE']

verification_status
['Source Verified' 'Not Verified' 'Verified']

issue_d
['Feb-2017' 'Apr-2016' 'Jul-2017' 'Oct-2016' 'Mar-2017' 'May-2016'
 'Mar-2016' 'Aug-2017' 'Jul-2016' 'Jan-2017' 'Sep-2016' 'Aug-2016'
 'Nov-2016' 'Dec-2016' 'Jun-2016' 'Sep-2017' 'Feb-2016' 'May-2017'
 'Jun-2017' 'Apr-2017' 'Jan-2016']

loan_status
['Current' 'Fully Paid' 'Late (16-30 days)' 'Charged Off'
 'La

In [96]:
# Kolom/feature yang harus di cleaning
col_need_to_clean = ['term', 'emp_length', 'issue_d', 'earliest_cr_line', 'last_pymnt_d',
                    'next_pymnt_d', 'last_credit_pull_d']

In [97]:
#cek values unique pada kolom term
X_train['term'].unique()

array(['36 months', '60 months'], dtype=object)

In [98]:
#Menghilangkan 'months' pada kolom term
X_train['term'].str.replace('months', '')

,term
147577,36
505776,36
412425,36
618730,36
111143,36
...,...
150426,36
195135,36
182511,36
82596,36


In [99]:
#ubah tipe data kolom term menjadi numeric
X_train['term'] = pd.to_numeric(X_train['term'].str.replace('months', ''))

In [100]:
X_train['term']

,term
147577,36
505776,36
412425,36
618730,36
111143,36
...,...
150426,36
195135,36
182511,36
82596,36


In [101]:
# Cek values apa saja yang harus di cleaning
X_train['emp_length'].unique()

array(['4 years', '10+ years', '< 1 year', '5 years', '6 years',
       '7 years', '2 years', '3 years', '8 years', '1 year', nan,
       '9 years'], dtype=object)

In [102]:
X_train['emp_length'] = X_train['emp_length'].str.replace('\+ years', '')
X_train['emp_length'] = X_train['emp_length'].str.replace(' years', '')
X_train['emp_length'] = X_train['emp_length'].str.replace('< 1 year', str(0))
X_train['emp_length'] = X_train['emp_length'].str.replace(' year', '')
# The line below is causing the error, it's trying to convert '10+' to a number
#X_train['emp_length'] = pd.to_numeric(X_train['emp_length'])

# Instead of directly converting to numeric, replace '10+' with 10 first:
X_train['emp_length'] = X_train['emp_length'].str.replace('10+', '10')
X_train['emp_length'].fillna(value=0, inplace=True) # Fill NaNs with 0
X_train['emp_length'] = pd.to_numeric(X_train['emp_length']) # Now convert to numeric

In [103]:
X_train['emp_length']

,emp_length
147577,4
505776,10
412425,0
618730,5
111143,6
...,...
150426,8
195135,0
182511,10
82596,3


In [121]:
# Cek feature date
col_date = ['issue_d', 'earliest_cr_line', 'last_pymnt_d',
                    'next_pymnt_d', 'last_credit_pull_d']

X_train[col_date]

,issue_d,earliest_cr_line,last_pymnt_d,next_pymnt_d,last_credit_pull_d
147577,2017-02-01,1994-08-01,2017-12-01,2018-01-01,2017-12-01
505776,2016-04-01,2001-09-01,2017-07-01,NaT,2017-11-01
412425,2017-07-01,1999-05-01,2017-12-01,2018-01-01,2017-12-01
618730,2016-10-01,2005-06-01,2017-12-01,2018-01-01,2017-12-01
111143,2017-03-01,2012-01-01,2017-12-01,2018-01-01,2017-12-01
...,...,...,...,...,...
150426,2017-02-01,1998-11-01,2017-11-01,NaT,2017-12-01
195135,2017-01-01,1993-09-01,2017-12-01,2018-01-01,2017-12-01
182511,2017-01-01,1988-03-01,2017-12-01,2018-01-01,2017-12-01
82596,2017-04-01,2003-01-01,2017-12-01,2018-01-01,2017-12-01


In [103]:
#mengubah kolom 'issue_d' menjadi tipe data date
X_train['issue_d'] = pd.to_datetime(X_train['issue_d'])

In [105]:
for col in col_date:
    X_train[col] = pd.to_datetime(X_train[col])

In [109]:
X_train[col_need_to_clean].info()

<class 'pandas.core.frame.DataFrame'>
Index: 607470 entries, 147577 to 165904
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   term                607470 non-null  int64         
 1   emp_length          607470 non-null  int64         
 2   issue_d             607470 non-null  datetime64[ns]
 3   earliest_cr_line    607470 non-null  datetime64[ns]
 4   last_pymnt_d        606735 non-null  datetime64[ns]
 5   next_pymnt_d        473147 non-null  datetime64[ns]
 6   last_credit_pull_d  607453 non-null  datetime64[ns]
dtypes: datetime64[ns](5), int64(2)
memory usage: 37.1 MB


In [123]:
for col in col_date:
    X_test[col] = pd.to_datetime(X_test[col])

In [124]:
X_test[col_need_to_clean].info()

<class 'pandas.core.frame.DataFrame'>
Index: 151868 entries, 280647 to 267584
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   term                151868 non-null  int64         
 1   emp_length          151868 non-null  int64         
 2   issue_d             151868 non-null  datetime64[ns]
 3   earliest_cr_line    151868 non-null  datetime64[ns]
 4   last_pymnt_d        151655 non-null  datetime64[ns]
 5   next_pymnt_d        118276 non-null  datetime64[ns]
 6   last_credit_pull_d  151867 non-null  datetime64[ns]
dtypes: datetime64[ns](5), int64(2)
memory usage: 9.3 MB
